# High-Fidelity Agent Evaluation with TraitBasis + Together Evals

## Overview
This notebook demonstrates how to generate realistic, multi-turn user interactions using TraitBasis and evaluate AI agent responses with Together Evals. 

### Why do I need this?
With different user personas, you can test your service from multiple perspectives and ensure every user gets high-quality responses, regardless of their traits.


By the end of this tutorial, you'll understand how to:
- Generate conversations with configurable user personas (traits like impatience, confusion, skepticism)
- Run simulations with Together AI models as agents
- Automatically evaluate agent performance using LLM-as-a-Judge
- Analyze results with scores, pass/fail status, and rationales

### What is TraitBasis?
TraitBasis is a method for controllable generation of user interactions that maintains consistent personas across multiple turns without suffering from persona drift or intent loss in long contexts.

### Workflow
1. **Configure**: Set up agent model, TraitBasis personas, and evaluation parameters
2. **Generate**: Create multi-turn conversations with realistic user behaviors
3. **Evaluate**: Upload to Together Evals and judge agent responses
4. **Analyze**: Review scores, rationales, and persona-specific performance

## 🚀 Setup and Installation


In [ ]:
import json
import time
from pathlib import Path
from datetime import datetime

import nest_asyncio
import together
from collinear.client import Client

nest_asyncio.apply()


## 🛠️ Utility Functions

Helper functions for formatting conversations, extracting personas, and handling results.


In [ ]:
def conversation_lines(messages):
    """Format message list into readable lines with role prefixes."""
    lines = []
    for message in messages:
        role = message.get('role')
        content = message.get('content')
        if content:
            lines.append(f"{role}: {content}")
    return lines


def conversation_text(messages):
    """Convert messages to plain text transcript."""
    return "\n".join(conversation_lines(messages))


def persona_from_traitmix(runner, traitmix):
    """Extract persona characteristics and traits from TraitMix config."""
    if not traitmix:
        return {}
    try:
        characteristics = runner._user_characteristics_payload(traitmix)
    except Exception:
        characteristics = {}
    return {
        'characteristics': characteristics or {},
        'traits': dict(getattr(traitmix, 'traits', {}) or {}),
    }


def print_evaluation_results(path: Path) -> None:
    """Parse and display evaluation results from JSONL file."""
    for idx, line in enumerate(path.read_text(encoding='utf-8').splitlines(), start=1):
        try:
            row = json.loads(line)
        except json.JSONDecodeError:
            print(f"[{idx}] could not parse result")
            print(line)
            continue
        score = row.get('score', '-')
        passed = row.get('pass')
        rationale = row.get('feedback') or row.get('rationale')
        print(f"[{idx}] score={score} status={(passed if passed is not None else '-')}")
        if rationale:
            print(f"  rationale: {rationale}")


def _needs_fallback(response: str) -> bool:
    """Check if agent response is empty, error, or stop signal."""
    if not response:
        return True
    stripped = response.strip()
    if not stripped:
        return True
    if stripped == "###STOP###":
        return True
    if stripped.lower().startswith("assistant returned empty response"):
        return True
    if stripped.lower().startswith("error:"):
        return True
    return False


## 🗝️ How to Get API Keys for Together AI and Collinear

To run these simulations, you'll need API keys for both Together AI and Collinear.

**Together AI:**
1. **Register for a Together AI account:**  
   Visit [Together AI](https://www.together.ai/) and sign up for a free account.
2. **Get your API key:**  
   After logging in, navigate to your account dashboard to find your API key.
3. **Set the environment variable:**  
   Make sure your API key is set in your shell environment as `TOGETHER_API_KEY`.  
   For example, in bash/zsh:
   ```
   export TOGETHER_API_KEY="your-together-api-key-here"
   export TOGETHER_BASE_URL="https://api.together.xyz/v1"
   ```

**Collinear:**
1. **Register for a Collinear AI account:**
   - Visit [Collinear AI](https://platform.collinear.ai/) and sign up for a free account.
2. **Access your API key:**
   - After logging in, navigate to the **Developer** menu in the bottom-left corner of the dashboard.
   - Copy your API key from there.
3. **Set the environment variable:**
   ```
   export COLLINEAR_API_KEY="your-collinear-api-key-here"
   ```

You'll need to have both API keys set (or configured in `configs/simulation_config.json`) before starting the simulation.


## 🔧 Configure Your Models

Before running the simulation, you need to configure the models in `configs/simulation_config.json`:

1. **`assistant_model_name`**: The model that will act as your assistant/agent in the conversations.
   - Default: `"openai/gpt-oss-20b"`
   - This should be a model available through Together AI's serverless API: https://docs.together.ai/docs/serverless-models

2. **`judge_model_name`**: The model that will evaluate the quality of the assistant's responses.
   - Default: `""deepseek-ai/DeepSeek-V3.1""`
   - This model should also be from Together AI's available models: https://docs.together.ai/docs/serverless-models

Make sure both models are set correctly in your configuration file before proceeding.




## ⚙️ Load Configuration

Loads simulation parameters, TraitMix persona configs, and Together Evals settings from JSON files.

**Key Configuration Sections:**
- `client`: Agent model, API keys, and connection settings
- `simulate`: Number of samples, conversation turns, temperature, concurrency
- `assess`: Judge model configuration
- `together`: Evaluation type, scoring thresholds, output paths


In [ ]:
CONFIG_DIR = Path('configs')
SIMULATION_CONFIG_FILE = CONFIG_DIR / 'simulation_config.json'
config = json.loads(SIMULATION_CONFIG_FILE.read_text())

# Load TraitMix persona configuration
traitmix_name = config.get('traitmix_config_file', 'traitmix_config_airline.json')
TRAITMIX_CONFIG_FILE = CONFIG_DIR / Path(traitmix_name).name
traitmix_config = json.loads(TRAITMIX_CONFIG_FILE.read_text())
TRAITMIX_TASKS = traitmix_config.get('tasks') or []

# Agent model and API settings
client_cfg = config.get('client', {}) or {}
CLIENT_ASSISTANT_MODEL_URL = client_cfg.get('assistant_model_url', 'https://api.together.xyz/v1')
CLIENT_ASSISTANT_MODEL_API_KEY = client_cfg.get('assistant_model_api_key')
CLIENT_ASSISTANT_MODEL_NAME = client_cfg.get('assistant_model_name', 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo')
CLIENT_COLLINEAR_API_KEY = client_cfg.get('collinear_api_key', 'demo-001')
CLIENT_TIMEOUT = float(client_cfg.get('timeout', 120))
CLIENT_MAX_RETRIES = int(client_cfg.get('max_retries', 3))
CLIENT_RATE_LIMIT_RETRIES = int(client_cfg.get('rate_limit_retries', 6))

# Simulation parameters
sim_cfg = config.get('simulate', {}) or {}
SIM_SAMPLES = sim_cfg.get('k', 3)  # Number of conversations to generate
SIM_EXCHANGES = sim_cfg.get('num_exchanges', 2)  # Turns per conversation
SIM_DELAY = sim_cfg.get('batch_delay', 0.2)
SIM_TRAITMIX_TEMPERATURE = sim_cfg.get('traitmix_temperature', 0.7)
SIM_TRAITMIX_MAX_TOKENS = sim_cfg.get('traitmix_max_tokens', 256)
SIM_MIX_TRAITS = bool(sim_cfg.get('mix_traits', False))  # Combine multiple traits
SIM_MAX_CONCURRENCY = int(sim_cfg.get('max_concurrency', 8))

# Judge model for evaluation
assess_cfg = config.get('assess', {}) or {}
ASSESS_JUDGE_MODEL_NAME = assess_cfg.get('judge_model_name')

# Together Evals configuration
together_cfg = config.get('together', {}) or {}
RESULTS_DIR = Path(together_cfg.get('output_directory', '.')).joinpath('results')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
JUDGE_SYSTEM_PROMPT = Path(together_cfg.get('judge_system_prompt', 'configs/judge_system_prompt.jinja')).read_text(encoding='utf-8')
TOGETHER_UPLOAD_PURPOSE = together_cfg.get('upload_purpose', 'eval')
TOGETHER_EVAL_TYPE = together_cfg.get('evaluation_type', 'score')
TOGETHER_MODEL_TO_EVALUATE = together_cfg.get('model_to_evaluate', 'assistant_response')
TOGETHER_JUDGE_MODEL_SOURCE = together_cfg.get('judge_model_source', 'serverless')
TOGETHER_MIN_SCORE = together_cfg.get('min_score', 1.0)
TOGETHER_MAX_SCORE = together_cfg.get('max_score', 10.0)
TOGETHER_PASS_THRESHOLD = together_cfg.get('pass_threshold', 7.0)
TOGETHER_POLL_TIMEOUT_SECONDS = int(together_cfg.get('poll_timeout_seconds', 300))
TOGETHER_POLL_INTERVAL_SECONDS = int(together_cfg.get('poll_interval_seconds', 5))
raw_prefix = together_cfg.get('results_filename_prefix') or together_cfg.get('output_filename') or 'together_eval'
FILENAME_BASE = (str(raw_prefix).rsplit('.', 1)[0]).rstrip('_')
RUN_ID = datetime.now().strftime('%Y%m%d_%H%M%S')

print(f'Loaded simulation config {SIMULATION_CONFIG_FILE}')
print(f'TraitMix config {TRAITMIX_CONFIG_FILE} | tasks: {TRAITMIX_TASKS or "<none>"}')


## 🎭 Understanding TraitMix Configuration

The TraitMix config defines user personas and behaviors for simulations.

**Key Components:**
- **Demographics**: Ages, genders, occupations, locations, languages
- **Traits**: Behavioral characteristics with intensity levels (0-2)
  - `impatience`: User wants quick answers, gets frustrated with delays
  - `confusion`: User struggles to articulate needs clearly
  - `skeptical`: User questions or doubts agent responses
  - `incoherence`: User messages lack clarity or logical flow
- **Intents**: User goals (e.g., track order, apply promo code, return item)
- **Tasks**: Domain-specific scenarios (e.g., "retail support", "airline booking")

TraitBasis samples from these distributions to create diverse, realistic user interactions.


In [ ]:
import os

# Set environment variables for API access
if CLIENT_ASSISTANT_MODEL_API_KEY:
    os.environ['TOGETHER_API_KEY'] = CLIENT_ASSISTANT_MODEL_API_KEY
if CLIENT_ASSISTANT_MODEL_URL:
    os.environ['TOGETHER_BASE_URL'] = CLIENT_ASSISTANT_MODEL_URL
if CLIENT_COLLINEAR_API_KEY:
    os.environ['COLLINEAR_API_KEY'] = CLIENT_COLLINEAR_API_KEY


## 🔌 Initialize Client

Creates the Collinear client with configured agent model and TraitBasis API credentials.

The client handles:
- Connection to your agent model (e.g., Together AI)
- TraitBasis API for persona-driven user simulation
- Rate limiting and retry logic


In [ ]:
if not CLIENT_ASSISTANT_MODEL_API_KEY:
    raise RuntimeError('assistant_model_api_key must be set in configs/simulation_config.json')

client = Client(
    assistant_model_url=CLIENT_ASSISTANT_MODEL_URL,
    assistant_model_api_key=CLIENT_ASSISTANT_MODEL_API_KEY,
    assistant_model_name=CLIENT_ASSISTANT_MODEL_NAME,
    collinear_api_key=CLIENT_COLLINEAR_API_KEY,
    timeout=CLIENT_TIMEOUT,
    max_retries=CLIENT_MAX_RETRIES,
    rate_limit_retries=CLIENT_RATE_LIMIT_RETRIES,
)

runner = client.simulation_runner


## 💬 Generate Simulated Conversations

Runs TraitBasis simulations to create realistic multi-turn conversations between users and your agent.

**What Happens Here:**
1. TraitBasis generates user messages with configured personas (traits, demographics, intents)
2. Your agent model responds to each user message
3. Conversations continue for the specified number of exchanges
4. Results are saved as JSONL with conversation history, agent response, and persona details

**Fallback Logic:**
If the agent returns an empty or error response, the code extracts the last valid assistant message from the conversation history.


In [ ]:
# Generate multi-turn conversations with TraitBasis personas
simulations = client.simulate(
    traitmix_config=traitmix_config,
    k=SIM_SAMPLES,
    num_exchanges=SIM_EXCHANGES,
    batch_delay=SIM_DELAY,
    traitmix_temperature=SIM_TRAITMIX_TEMPERATURE,
    traitmix_max_tokens=SIM_TRAITMIX_MAX_TOKENS,
    mix_traits=SIM_MIX_TRAITS,
    max_concurrency=SIM_MAX_CONCURRENCY,
)

# Process simulation results and handle fallback responses
rows = []
for sim in simulations:
    messages = list(sim.conv_prefix)
    assistant_response = (sim.response or "").strip()

    # If final response is empty/error, use last valid assistant message
    if _needs_fallback(assistant_response):
        fallback = ""
        cutoff_index = None
        # Search backwards for last valid assistant response
        for idx in range(len(messages) - 1, -1, -1):
            message = messages[idx]
            if message.get("role") == "assistant":
                candidate = (message.get("content") or "").strip()
                if candidate and "###STOP###" not in candidate:
                    fallback = candidate
                    cutoff_index = idx
                    break
        if fallback:
            assistant_response = fallback
            if cutoff_index is not None:
                messages = messages[: cutoff_index + 1]

    rows.append(
        {
            "conversation_messages": messages,
            "assistant_response": assistant_response,
            "traitmix_persona": persona_from_traitmix(runner, getattr(sim, "traitmix", None)),
        }
    )

# Save dataset as JSONL for Together Evals
dataset_path = RESULTS_DIR / f"{FILENAME_BASE}_{RUN_ID}_dataset.jsonl"
with dataset_path.open("w", encoding="utf-8") as fh:
    for row in rows:
        convo_lines = conversation_lines(row["conversation_messages"])
        serializable = {
            "conversation": "\n".join(convo_lines),
            "assistant_response": row["assistant_response"],
            "traitmix_persona": row["traitmix_persona"],
        }
        fh.write(json.dumps(serializable, ensure_ascii=False))
        fh.write(chr(10))
print(f"Saved {len(rows)} simulations to {dataset_path}")

for idx, row in enumerate(rows[:5], start=1):
    print()
    print("=" * 40)
    print(f"Conversation {idx}")
    print("-" * 40)
    persona = row["traitmix_persona"] or {}
    print("Persona:")
    print(json.dumps(persona, indent=2, ensure_ascii=False) if persona else "  <none>")
    print()
    print("Transcript:")
    turns = conversation_lines(row["conversation_messages"])
    for turn_no, line in enumerate(turns, start=1):
        print(f"{turn_no:02d}. {line}")
    print(f"{len(turns) + 1:02d}. assistant: {row['assistant_response'] or '<no response>'}")


## 📤 Upload Dataset & Start Evaluation

Uploads the conversation dataset to Together AI and creates an evaluation job.

**Evaluation Configuration:**
- **Judge Model**: The LLM that evaluates agent responses
- **Evaluation Type**: `score` (rates responses on a scale)
- **Score Range**: Min and max scores (e.g., 1-10)
- **Pass Threshold**: Minimum score considered successful
- **Judge System Prompt**: Custom instructions for the judge model. References the conversation via `{{conversation}}` template variable

- **Model to Evaluate**: Set to `assistant_response` (a column in our dataset containing pre-generated responses)
- **Note**: We use specialized Collinear models for generation before evaluation, rather than having Together generate responses during evaluation

The evaluation job runs asynchronously on Together's infrastructure.


In [ ]:
together_client = together.Together(api_key=CLIENT_ASSISTANT_MODEL_API_KEY)

# Upload dataset to Together AI
upload = together_client.files.upload(file=str(dataset_path), purpose=TOGETHER_UPLOAD_PURPOSE)
upload_id = getattr(upload, 'id', None)
if upload_id is None:
    upload_id = upload['id']

In [ ]:
evaluation_job = together_client.evaluation.create(
    type=TOGETHER_EVAL_TYPE,
    # Pass the detailed configuration object
    # We are evaluating the field 'assistant_response' with responses that we generated earlier.
    model_to_evaluate='assistant_response',
    input_data_file_path=upload_id,
    # Judge model details, it is better to use as strong model as possible
    judge_model=ASSESS_JUDGE_MODEL_NAME,
    judge_model_source=TOGETHER_JUDGE_MODEL_SOURCE,
    judge_system_template=JUDGE_SYSTEM_PROMPT,
    min_score=TOGETHER_MIN_SCORE,
    max_score=TOGETHER_MAX_SCORE,
    pass_threshold=TOGETHER_PASS_THRESHOLD
)

workflow_id = evaluation_job.workflow_id
print(f'Started evaluation {workflow_id}')

## 📊 Poll Results & Analysis

Waits for the evaluation to complete, downloads results, and displays detailed analysis.

**What You'll See:**
- Evaluation status updates during polling
- Per-conversation breakdown with:
  - **Persona**: Traits and characteristics for each simulated user
  - **Transcript**: Full conversation history
  - **Assessment**: Score, pass/fail status, and judge rationale

This allows you to understand how your agent performs across different user personas and identify areas for improvement.

You can also see your jobs statuses at https://api.together.ai/evaluations


In [ ]:
# Poll for evaluation completion
deadline = time.time() + TOGETHER_POLL_TIMEOUT_SECONDS
results_path = None
while time.time() < deadline:
    status_obj = together_client.evaluation.status(workflow_id)
    status_raw = str(getattr(status_obj, "status", "pending"))
    state = status_raw.lower().split('.')[-1]
    print(f"status: {status_raw}")
    
    # Check if evaluation is complete
    if state in {"completed", "success", "failed", "error", "user_error"}:
        results = getattr(status_obj, "results", None)
        if isinstance(results, dict) and results.get("result_file_id"):
            # Download results file
            results_path = RESULTS_DIR / f"{FILENAME_BASE}_{RUN_ID}_{workflow_id}_results.jsonl"
            together_client.files.retrieve_content(results["result_file_id"], output=str(results_path))
            print(f"Downloaded results to {results_path}")
            
            # Parse evaluation results
            evaluation_rows = []
            with results_path.open("r", encoding="utf-8") as fh:
                for line in fh:
                    text = line.strip()
                    if text:
                        evaluation_rows.append(json.loads(text))
            
            # Display results with personas and transcripts
            for idx, row in enumerate(rows, start=1):
                evaluation = evaluation_rows[idx - 1] if idx - 1 < len(evaluation_rows) else {}
                print()
                print("=" * 40)
                print(f"Conversation {idx}")
                print("-" * 40)
                persona = row.get("traitmix_persona") or {}
                print("Persona:")
                print(json.dumps(persona, indent=2, ensure_ascii=False) if persona else "  <none>")
                print()
                print("Transcript:")
                turns = conversation_lines(row.get("conversation_messages", []))
                for turn_no, line in enumerate(turns, start=1):
                    print(f"{turn_no:02d}. {line}")
                print(f"{len(turns) + 1:02d}. assistant: {row.get('assistant_response') or '<no response>'}")
                score = evaluation.get("score", "-")
                passed = evaluation.get("pass")
                rationale = evaluation.get("feedback") or evaluation.get("rationale")
                print()
                print("Assessment:")
                print(f"  score: {score}")
                if passed is not None:
                    print(f"  pass: {passed}")
                if rationale:
                    print(f"  rationale: {rationale}")
        break
    time.sleep(TOGETHER_POLL_INTERVAL_SECONDS)
else:
    print("Timed out waiting for evaluation to finish.")


We can see that for some user types the assistant score is higher than for others. This information can be used to create improved assistant responses for these examples and fine-tune the model to improve on them.

In [ ]:
results = together_client.evaluation.status(workflow_id).results

Results Summary

The `results` object contains aggregated evaluation metrics:
- `mean_score`: Average score across all evaluated conversations (1-5 scale)
- `pass_percentage`: Percentage of conversations that met the pass threshold (≥4.0)
- `std_score`: Standard deviation of scores, indicating consistency
- `failed_samples`: Number of samples that failed to evaluate
- `generation_fail_count`: Number of failures during response generation
- `invalid_score_count`: Number of evaluations with invalid scores
- `judge_fail_count`: Number of failures during judge evaluation
- `result_file_id`: File ID containing detailed per-conversation results with judge feedback.


In [ ]:
print(results)

## Summary

This notebook demonstrates how to:
1. Generate synthetic user-assistant conversations using TraitMix personas
2. Evaluate assistant performance using Together AI's evaluation platform
3. Analyze results to understand system behavior across diverse user types

By simulating conversations with varied user personas and systematically evaluating responses,
you can assess how your AI systems will behave for different users, track improvements over time,
and gain confidence in your results before deploying to production. This workflow enables data-driven
iteration on prompts, models, and system design.